In [110]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using CSV
using JLD
using DataFrames
using JSON
using MAT

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)


importing packages...
packages and functions imported
Elapsed time = 0.038923025131225586 seconds 
 


In [111]:
### Set parameters

betti_colors = [["#243a4c"] ["#406372"] ["#66939e"] ["#9bc3c6"]]

# Read from config file
config_file = "config092420.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
betti_files = filter(x->!occursin("dsi", x), betti_files)

println("Located the following graph files:")
# for betti_file in betti_files
#     println(betti_file)
# end


# Locate the nametags
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


for nametag in nametags
    namefiles = filter(x->occursin(nametag,x), betti_files)
    if length(namefiles)<12
    println(nametag)
    println(length(namefiles))
    end
end

nametags

Located the following graph files:
IID_70nodes_200reps_092420_forward_bettis.jld
IID_70nodes_200reps_092420_overlap_a01_b03_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a02_b04_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a03_b05_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a04_b06_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a05_b07_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a06_b08_overlap_bettis.jld
IID_70nodes_200reps_092420_overlap_a07_b09_overlap_bettis.jld
IID_70nodes_200reps_092420_thresh015_edge363_noiseOnly_bettis.jld
IID_70nodes_200reps_092420_thresh015_edge363_randomized_threshold_bettis.jld
IID_70nodes_200reps_092420_thresh015_edge363_threshold_bettis.jld
IID_70nodes_200reps_092420_thresh01_edge242_noiseOnly_bettis.jld
IID_70nodes_200reps_092420_thresh01_edge242_randomized_threshold_bettis.jld
IID_70nodes_200reps_092420_thresh01_edge242_threshold_bettis.jld
IID_70nodes_200reps_092420_thresh025_edge604_noiseOnly_bettis.jld
IID_

clique10_70nodes_200reps_10_092420_thresh08_edge1932_threshold_bettis.jld
clique10_70nodes_200reps_10_092420_thresh09_edge2174_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh015_edge363_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh01_edge242_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh025_edge604_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh02_edge483_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh035_edge846_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh03_edge725_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh045_edge1087_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh04_edge966_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh055_edge1329_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh05_edge1208_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thresh065_edge1570_threshold_bettis.jld
clique15_70nodes_200reps_15_092420_thre

coreperiphery_70nodes_200reps_15_5_10_5_092420_overlap_a03_b05_overlap_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_overlap_a04_b06_overlap_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_overlap_a05_b07_overlap_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_overlap_a06_b08_overlap_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_overlap_a07_b09_overlap_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh015_edge363_noiseOnly_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh015_edge363_randomized_threshold_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh015_edge363_threshold_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh01_edge242_noiseOnly_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh01_edge242_randomized_threshold_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh01_edge242_threshold_bettis.jld
coreperiphery_70nodes_200reps_15_5_10_5_092420_thresh02

disassortative_70nodes_200reps_20_5_10_5_092420_overlap_a05_b07_overlap_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_overlap_a06_b08_overlap_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_overlap_a07_b09_overlap_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh015_edge363_noiseOnly_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh015_edge363_randomized_threshold_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh015_edge363_threshold_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh01_edge242_noiseOnly_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh01_edge242_randomized_threshold_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh01_edge242_threshold_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh025_edge604_noiseOnly_bettis.jld
disassortative_70nodes_200reps_20_5_10_5_092420_thresh025_edge604_randomized_threshold_bettis.jld
disassortative_70nodes_20

dotProduct_70nodes_200reps_3_092420_forward_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a01_b03_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a02_b04_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a03_b05_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a04_b06_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a05_b07_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a06_b08_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_overlap_a07_b09_overlap_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh015_edge363_noiseOnly_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh015_edge363_randomized_threshold_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh015_edge363_threshold_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh01_edge242_noiseOnly_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh01_edge242_randomized_threshold_bettis.jld
dotProduct_70nodes_200reps_3_092420_thresh01_edge242_

randomGeom_70nodes_200reps_3_092420_thresh015_edge363_randomized_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh015_edge363_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh01_edge242_noiseOnly_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh01_edge242_randomized_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh01_edge242_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh025_edge604_noiseOnly_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh025_edge604_randomized_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh025_edge604_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh02_edge483_noiseOnly_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh02_edge483_randomized_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh02_edge483_threshold_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh035_edge846_noiseOnly_bettis.jld
randomGeom_70nodes_200reps_3_092420_thresh035_edge846_randomiz

rmsd_70nodes_200reps_3_092420_thresh045_edge1087_noiseOnly_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh045_edge1087_randomized_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh045_edge1087_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh04_edge966_noiseOnly_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh04_edge966_randomized_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh04_edge966_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh055_edge1329_noiseOnly_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh055_edge1329_randomized_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh055_edge1329_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh05_edge1208_noiseOnly_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh05_edge1208_randomized_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh05_edge1208_threshold_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh065_edge1570_noiseOnly_bettis.jld
rmsd_70nodes_200reps_3_092420_thresh065_edge1570_random

sqdEuclidean_70nodes_200reps_3_092420_thresh03_edge725_noiseOnly_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh03_edge725_randomized_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh03_edge725_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh045_edge1087_noiseOnly_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh045_edge1087_randomized_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh045_edge1087_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh04_edge966_noiseOnly_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh04_edge966_randomized_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh04_edge966_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh055_edge1329_noiseOnly_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh055_edge1329_randomized_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_092420_thresh055_edge1329_threshold_bettis.jld
sqdEuclidean_70nodes_200reps_3_0

wprobTriangle5_70nodes_200reps_05_092420_thresh04_edge966_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh055_edge1329_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh05_edge1208_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh065_edge1570_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh06_edge1449_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh075_edge1812_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh07_edge1691_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh085_edge2053_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh08_edge1932_threshold_bettis.jld
wprobTriangle5_70nodes_200reps_05_092420_thresh09_edge2174_threshold_bettis.jld
wprobTriangle6_70nodes_200reps_06_092420_thresh015_edge363_threshold_bettis.jld
wprobTriangle6_70nodes_200reps_06_092420_thresh01_edge242_threshold_bettis.jld
wprobTriangle6_70nodes_200reps_06_0924

59-element Array{Any,1}:
 "forward"                                
 "overlap_a01_b03_overlap"                
 "overlap_a02_b04_overlap"                
 "overlap_a03_b05_overlap"                
 "overlap_a04_b06_overlap"                
 "overlap_a05_b07_overlap"                
 "overlap_a06_b08_overlap"                
 "overlap_a07_b09_overlap"                
 "thresh015_edge363_noiseOnly"            
 "thresh015_edge363_randomized_threshold" 
 "thresh015_edge363_threshold"            
 "thresh01_edge242_noiseOnly"             
 "thresh01_edge242_randomized_threshold"  
 ⋮                                        
 "thresh07_edge1691_noiseOnly"            
 "thresh07_edge1691_randomized_threshold" 
 "thresh07_edge1691_threshold"            
 "thresh085_edge2053_noiseOnly"           
 "thresh085_edge2053_randomized_threshold"
 "thresh085_edge2053_threshold"           
 "thresh08_edge1932_noiseOnly"            
 "thresh08_edge1932_randomized_threshold" 
 "thresh08_edge1932_threshold

In [119]:

####### Writing for JSON
using JSON

## Filter to include JUST the threshold ones
thresh_files = filter(x -> occursin("threshold",x),betti_files)
thresh_files = filter(x -> !occursin("randomized",x),thresh_files)
thresh_files = filter(x -> !occursin("clique",x),thresh_files)
thresh_files = filter(x -> !occursin("Triangle",x),thresh_files)

model_names = []
for thresh_file in thresh_files
    name = split(thresh_file, "_")[1]
    model_names = [model_names; name]
end

unique!(model_names)

big_dict = Dict()
df = DataFrame(edge = collect(1:binomial(NNODES,2)))

for model_name in model_names



    model_dict = Dict()

    # Run through all the threshold files and extract Betti curves

    model_thresh_files = filter(x->startswith(x,model_name), thresh_files)

    

    for model_thresh_file in model_thresh_files
        

        rho_dict = Dict()

        bettisArray = load("../processed_data/results/70nodes/$(model_thresh_file)","bettisArray")
        bettisArrayAvg = dropdims(mean(bettisArray, dims=1), dims=1)
        bettisArrayStd = dropdims(std(bettisArray,dims=1), dims=1)
        
        # Extract the rho value or edge number and add to dictionary
        thresh_edge = parse(Int,split(split(model_thresh_file,"edge")[2], "_")[1])
        rho = replace(split(split(model_thresh_file,"_edge")[1], "_thresh")[2], "."=> "")


        for dim in 1:MAXDIM
            rho_dict["dim$(dim)"] = bettisArrayAvg[:,dim]
            rho_dict["std$(dim)"] = bettisArrayStd[:, dim]
            df[!, "$(model_name)_thresh$(rho)_dim$(dim)"] = bettisArrayAvg[:, dim]
        end




#         model_dict["$(thresh_edge)"] = rho_dict

    end


#     big_dict["$(model_name)"] = model_dict
    
    println("finished $(model_name)")
    
end

big_dict





df

finished IID
finished assortative
finished coreperiphery
finished cosineGeometric
finished disassortative
finished discreteUniformConf
finished dotProduct
finished geometricConf
finished randomGeom
finished ringLattice
finished rmsd
finished sqdEuclidean


,edge,IID_thresh015_dim1,IID_thresh015_dim2,IID_thresh015_dim3,IID_thresh015_dim4
,Int64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0


In [120]:
CSV.write("test_bettis.csv",df)

"test_bettis.csv"

In [ ]:
## Write to json
open("../templates/foo4.json", "w") do f
    JSON.print(f, big_dict)
end
println("done saving")

## Loading and wrapping classification mat files


In [20]:
model_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform",
    "dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ]
mat_dir = "../processed_data/classification/"
mat_files = readdir(mat_dir)

17-element Array{String,1}:
 "matrix_prenoise_0.1.mat" 
 "matrix_prenoise_0.15.mat"
 "matrix_prenoise_0.2.mat" 
 "matrix_prenoise_0.25.mat"
 "matrix_prenoise_0.3.mat" 
 "matrix_prenoise_0.35.mat"
 "matrix_prenoise_0.4.mat" 
 "matrix_prenoise_0.45.mat"
 "matrix_prenoise_0.5.mat" 
 "matrix_prenoise_0.55.mat"
 "matrix_prenoise_0.6.mat" 
 "matrix_prenoise_0.65.mat"
 "matrix_prenoise_0.7.mat" 
 "matrix_prenoise_0.75.mat"
 "matrix_prenoise_0.8.mat" 
 "matrix_prenoise_0.85.mat"
 "matrix_prenoise_0.9.mat" 

In [19]:
# Make a n_threhsolds x n_models^2 matrix

mat1 = matread("$(mat_dir)/$(mat_files[1])")
mat1["C"]

12×12 Array{Float64,2}:
 61.0    0.0   0.0   0.0  39.0   0.0   0.0  0.0  0.0    0.0   0.0   0.0
  0.0  100.0   0.0   0.0   0.0   0.0   0.0  0.0  0.0    0.0   0.0   0.0
 73.0    0.0  27.0   0.0   0.0   0.0   0.0  0.0  0.0    0.0   0.0   0.0
  0.0    0.0   0.0  97.0   0.0   0.0   0.0  0.0  0.0    0.0   3.0   0.0
 27.0    0.0   0.0   0.0  73.0   0.0   0.0  0.0  0.0    0.0   0.0   0.0
  0.0    0.0   0.0   1.0   0.0  89.0   0.0  0.0  1.0    0.0   4.0   5.0
  0.0    0.0   0.0   2.0   0.0   0.0  98.0  0.0  0.0    0.0   0.0   0.0
  0.0    0.0   0.0  62.0   0.0   0.0  29.0  6.0  0.0    0.0   1.0   2.0
  0.0    0.0   0.0  64.0   0.0   1.0   0.0  0.0  1.0    0.0  17.0  17.0
  0.0    0.0   0.0   0.0   0.0   0.0   0.0  0.0  0.0  100.0   0.0   0.0
  0.0    0.0   0.0  65.0   0.0   0.0   0.0  0.0  0.0    0.0  25.0  10.0
  0.0    0.0   0.0  61.0   0.0   1.0   0.0  0.0  0.0    0.0  21.0  17.0

In [23]:
col_names = ["$(a)_$(b)" for a in model_names for b in model_names]

144-element Array{String,1}:
 "IID_IID"                          
 "IID_assortative"                  
 "IID_coreperiphery"                
 "IID_cosineGeometric"              
 "IID_disassortative"               
 "IID_discreteUniform"              
 "IID_dotProduct"                   
 "IID_geometricConf"                
 "IID_randomGeometric"              
 "IID_ringLattice"                  
 "IID_rmsd"                         
 "IID_squaredEuclidean"             
 "assortative_IID"                  
 ⋮                                  
 "squaredEuclidean_IID"             
 "squaredEuclidean_assortative"     
 "squaredEuclidean_coreperiphery"   
 "squaredEuclidean_cosineGeometric" 
 "squaredEuclidean_disassortative"  
 "squaredEuclidean_discreteUniform" 
 "squaredEuclidean_dotProduct"      
 "squaredEuclidean_geometricConf"   
 "squaredEuclidean_randomGeometric" 
 "squaredEuclidean_ringLattice"     
 "squaredEuclidean_rmsd"            
 "squaredEuclidean_squaredEuclidean"

In [41]:

df = DataFrame(mat1["C"])
rename!(df,Symbol.(model_names))
df.true_class = model_names
df

,IID,assortative,coreperiphery,cosineGeometric,disassortative,discreteUniform
,Float64,Float64,Float64,Float64,Float64,Float64
1,61.0,0.0,0.0,0.0,39.0,0.0
2,0.0,100.0,0.0,0.0,0.0,0.0
3,73.0,0.0,27.0,0.0,0.0,0.0
4,0.0,0.0,0.0,97.0,0.0,0.0
5,27.0,0.0,0.0,0.0,73.0,0.0
6,0.0,0.0,0.0,1.0,0.0,89.0
7,0.0,0.0,0.0,2.0,0.0,0.0
8,0.0,0.0,0.0,62.0,0.0,0.0
9,0.0,0.0,0.0,64.0,0.0,1.0


In [43]:
df2 = stack(df,1:12)
rename!(df2, [:trueClass, :predictedClass, :accuracy])

,trueClass,predictedClass,accuracy
,String,Cat…,Float64
1,IID,IID,61.0
2,assortative,IID,0.0
3,coreperiphery,IID,73.0
4,cosineGeometric,IID,0.0
5,disassortative,IID,27.0
6,discreteUniform,IID,0.0
7,dotProduct,IID,0.0
8,geometricConf,IID,0.0
9,randomGeometric,IID,0.0


In [52]:
df2.truePredicted = ["$(b)_$(a)" for a in model_names for b in model_names]
df2

,trueClass,predictedClass,accuracy,truePredicted
,String,Cat…,Float64,String
1,IID,IID,61.0,IID_IID
2,assortative,IID,0.0,assortative_IID
3,coreperiphery,IID,73.0,coreperiphery_IID
4,cosineGeometric,IID,0.0,cosineGeometric_IID
5,disassortative,IID,27.0,disassortative_IID
6,discreteUniform,IID,0.0,discreteUniform_IID
7,dotProduct,IID,0.0,dotProduct_IID
8,geometricConf,IID,0.0,geometricConf_IID
9,randomGeometric,IID,0.0,randomGeometric_IID


In [105]:
# Get the two vectors we care about
df3 = DataFrame(TP = df2.truePredicted)


,TP
,String
1,IID_IID
2,assortative_IID
3,coreperiphery_IID
4,cosineGeometric_IID
5,disassortative_IID
6,discreteUniform_IID
7,dotProduct_IID
8,geometricConf_IID
9,randomGeometric_IID


In [106]:
# repeat

for mat_file in mat_files
    mat = matread("$(mat_dir)/$(mat_file)")["C"]
    thresh = "thresh$(replace(split(split(mat_file, ".mat")[1], "_")[3], "."=>""))"
    df = stack(DataFrame(mat),1:12)
    println(thresh)
    df3[thresh] = df.value
end
df3

thresh01
thresh015
thresh02
thresh025
thresh03
thresh035
thresh04
thresh045
thresh05
thresh055
thresh06
thresh065
thresh07
thresh075
thresh08
thresh085
thresh09


┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[106]:8
└ @ Core ./In[106]:8


,TP,thresh01,thresh015,thresh02,thresh025,thresh03
,String,Float64,Float64,Float64,Float64,Float64
1,IID_IID,61.0,63.0,82.0,87.0,98.0
2,assortative_IID,0.0,0.0,0.0,0.0,0.0
3,coreperiphery_IID,73.0,1.0,0.0,0.0,0.0
4,cosineGeometric_IID,0.0,0.0,0.0,0.0,0.0
5,disassortative_IID,27.0,13.0,10.0,11.0,2.0
6,discreteUniform_IID,0.0,0.0,0.0,0.0,0.0
7,dotProduct_IID,0.0,0.0,0.0,0.0,0.0
8,geometricConf_IID,0.0,0.0,0.0,0.0,0.0
9,randomGeometric_IID,0.0,0.0,0.0,0.0,0.0


In [109]:
CSV.write("test.csv",df3)

"test.csv"